In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bbc_preprocessed.csv to bbc_preprocessed.csv


In [ ]:
import re
import nltk
import pandas as pd
from textblob import Word
import numpy as np
import csv
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers import  Dropout, Dense
from sklearn import metrics

Using TensorFlow backend.


In [ ]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
#preprocessed dataset as input - text cleaned, stopwords removed, lemmatized
data = pd.read_csv('bbc_preprocessed.csv')
#Label encoding
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'category'. 
data['category']= label_encoder.fit_transform(data['category']) 
data['category'].unique() 

x = data['text'].tolist()
y = data['category'].tolist()

#feature extraction using count vectorizer and tf-idf
cv=CountVectorizer()
word_count_vector=cv.fit_transform(x)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

vect = TfidfVectorizer(stop_words='english',min_df=2)
X = vect.fit_transform(x)
Y = np.array(y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print ("\ntrain size:", X_train.shape)
print ("\ntest size:", X_test.shape)


train size: (1056, 10238)

test size: (265, 10238)


In [ ]:
model_DNN = Build_Model_DNN_Text(X_train.shape[1], 5)
model_DNN.fit(X_train, Y_train,
                              validation_data=(X_test, Y_test),
                              epochs=20,
                              batch_size=128,
                              verbose=2)
model_predictions = model_DNN.predict(X_test)

Train on 1056 samples, validate on 265 samples
Epoch 1/20
 - 3s - loss: 1.4575 - acc: 0.3627 - val_loss: 1.3285 - val_acc: 0.7170
Epoch 2/20
 - 1s - loss: 1.1477 - acc: 0.4309 - val_loss: 1.0636 - val_acc: 0.3547
Epoch 3/20
 - 1s - loss: 0.8034 - acc: 0.5890 - val_loss: 0.5500 - val_acc: 0.7358
Epoch 4/20
 - 1s - loss: 0.4219 - acc: 0.7803 - val_loss: 0.4796 - val_acc: 0.7358
Epoch 5/20
 - 1s - loss: 0.3580 - acc: 0.7822 - val_loss: 0.4617 - val_acc: 0.7358
Epoch 6/20
 - 1s - loss: 0.3511 - acc: 0.7822 - val_loss: 0.4617 - val_acc: 0.7358
Epoch 7/20
 - 1s - loss: 0.3476 - acc: 0.7850 - val_loss: 0.4609 - val_acc: 0.7358
Epoch 8/20
 - 1s - loss: 0.3440 - acc: 0.8390 - val_loss: 0.4571 - val_acc: 0.8151
Epoch 9/20
 - 1s - loss: 0.3400 - acc: 0.8523 - val_loss: 0.4500 - val_acc: 0.8151
Epoch 10/20
 - 1s - loss: 0.3393 - acc: 0.8608 - val_loss: 0.4328 - val_acc: 0.8189
Epoch 11/20
 - 1s - loss: 0.3226 - acc: 0.8580 - val_loss: 0.4047 - val_acc: 0.8151
Epoch 12/20
 - 1s - loss: 0.2991 - acc

In [ ]:
model_DNN.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               5242368   
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)              

In [ ]:
loss, accuracy = model_DNN.evaluate(X_train, Y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_DNN.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
predictions = model_DNN.predict(X_test)
matrix = metrics.confusion_matrix(Y_test, predictions.argmax(axis=1))
print(matrix)

Training Accuracy: 1.0000
Testing Accuracy:  0.9698
[[100   0   1   0   0]
 [  0  16   1   0   1]
 [  1   0  23   0   1]
 [  0   0   0  94   0]
 [  2   0   1   0  24]]


In [ ]:
predicted = model_DNN.predict(X_test)
print(metrics.classification_report (Y_test, np.argmax(predicted, axis = 1)))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       102
           1       0.94      0.99      0.96        75
           2       0.96      0.94      0.95        97
           3       0.98      0.99      0.98        88
           4       0.99      0.98      0.98        83

    accuracy                           0.97       445
   macro avg       0.97      0.97      0.97       445
weighted avg       0.97      0.97      0.97       445

